## 2.1.3 tf.estimator

In [ ]:
import os
import tensorflow as tf 

from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer

import numpy as np
import sys

### 기본 데이터를 생성합니다.
 
 - 긍,부정에 대한 문장을 간략하게 생성하였고, 첫번째가 긍정, 두번째가 부정입니다.

In [ ]:
samples = ['너 오늘 이뻐보인다', 
           '나는 오늘 기분이 더러워', 
           '끝내주는데, 좋은 일이 있나봐', 
           '좋은 일이 생겼어', 
           '아 짜증나', 
           '환상적인데, 정말 좋은거 같아']

label = [[1], [0], [1], [1],[0], [1]]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

sequences = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=5, padding='post')

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print(word_index)
print(sequences)
print(label)

In [ ]:
VOCAB_SIZE = len(word_index) + 1 #문장의 단어 사이즈
EMB_SIZE = 128
NUM_EPOCHS = 3
BATCH_SIZE = 2

def mapping_fn(X, Y=None):
    input, label = {'x': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    dataset = dataset.batch(BATCH_SIZE)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [ ]:
def model_fn(features, labels, mode):
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    input_layer = keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features['x'])
    avg_pool_layer = keras.layers.GlobalAveragePooling1D()(input_layer)
    dense_layer_1 = keras.layers.Dense(16, activation=tf.nn.relu)(avg_pool_layer)
    dense_layer_2 = keras.layers.Dense(16, activation=tf.nn.relu)(dense_layer_1)
    output_layer = keras.layers.Dense(2)(dense_layer_2)

    one_hot_labels = tf.squeeze(tf.one_hot(labels, 2), axis=1)
    loss = tf.losses.softmax_cross_entropy(one_hot_labels, output_layer)

    if TRAIN:
        global_step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  train_op=train_op,
                  loss=loss)

In [ ]:
model_dir = "./data_out/checkpoint/tutorial/"

model_basic = tf.estimator.Estimator(model_fn, model_dir=model_dir)
model_basic.train(train_input_fn) 